In [2]:
import pandas as pd
import numpy as np

DIR_DATA = '/Users/mohanli/Documents/citadel/46924_Natual_Language_Processing-Applications_in_Finance/proj1/data/'

company_list = pd.read_csv(DIR_DATA + 'filtered_sp500.csv', index_col=0)
company_list = company_list[company_list['cik'].notna()]

In [3]:
company_list.head()

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret,cik,sic,naics,gind,gsubind
0,2011-01-31,60986,NEWELL RUBBERMAID INC,65122910,11,1,3089,NWL,7875,1,1989-04-27,2022-03-31,0.058856,814453.0,3990,3399,252010,25201050
1,2011-01-31,85914,BEST BUY COMPANY INC,8651610,11,1,5731,BBY,2184,1,1999-06-30,2022-03-31,-0.008457,764478.0,5731,443142,255040,25504020
2,2011-01-31,80711,APARTMENT INVESTMENT & MGMT CO,03748R10,18,1,6798,AIV,30490,1,2003-03-14,2020-12-18,-0.010836,922864.0,6798,531110,601010,60101060
3,2011-01-31,59176,AMERICAN EXPRESS CO,2581610,11,1,6141,AXP,1447,1,1976-07-01,2022-03-31,0.014912,4962.0,6141,522210,402020,40202010
9,2011-01-31,59408,BANK OF AMERICA CORP,6050510,11,1,6021,BAC,7647,1,1976-07-01,2022-03-31,0.029235,70858.0,6020,522110,401010,40101010


In [177]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        row = []
        for td in tr.find_all('td'):
            if td.find_all('a'):
                row.append(td.find_all('a')[0].get("href"))
            else:
                row.append(td.get_text(strip=True))
        rows.append(row)
    return rows

In [176]:
import html
import html2text

def parse_sec_text(text, documents):
    html_10q = None
    for document in documents:
        document_type = re.findall("<TYPE>(.*?)\n", document, re.DOTALL)[0]
        if document_type.replace('\t', '').lstrip().rstrip() == '10-K':
            html_10q = re.findall('(<(html|HTML).*?</(html|HTML)>)', document, re.DOTALL)[0][0]
            break
    else:
        print('Unable to find document 10-Q!')
        return None
    html_10q = html.unescape(html_10q)
    html_10q = re.sub('(<div style="display:none">.*?</div>)', '', html_10q)
    html_10q = re.sub('(<TABLE.*?</TABLE>)', '', html_10q)
    helper = html2text.HTML2Text()
    helper.ignore_links = True
    text_10q = helper.handle(re.findall('(<(body|BODY).*?</(body|BODY)>)', html_10q, re.DOTALL)[0][0])
    text_10q = re.sub('\\n', ' ', text_10q)
    # sec_header = re.findall("<SEC-HEADER>(.*?)</SEC-HEADER>", text, re.DOTALL)[0]
    # sic = re.findall('STANDARD INDUSTRIAL CLASSIFICATION:(.*?)\n', sec_header)[0].replace('\t', '').lstrip().rstrip()
    return text_10q

In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import datetime
import re

for cik_int in company_list:
    cik = str(cik_int)
    dataType = '10-K'
    dateb = '20220201'

    base_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={dataType}&dateb={dateb}"

    browser = webdriver.Chrome(executable_path=r"/opt/homebrew/bin/chromedriver")
    browser.get(base_url)
    html0 = browser.page_source

    soup0 = BeautifulSoup(html0, 'html.parser')
    
    try:
        tableData0 = tableDataText(soup0.findAll("table", {"summary": "Results"})[0])

        for row in tableData0[1:]:
            if datetime.datetime.strptime(row[3], '%Y-%m-%d').date() > datetime.date(2016, 1, 1):
                date = row[3]
                url = 'https://www.sec.gov' + row[1]

                browser.get(url)
                html1 = browser.page_source
                soup1 = BeautifulSoup(html1, 'html.parser')

                tableData1 = tableDataText(soup1.findAll("table", {"summary" : "Document Format Files"})[0])
                for row in tableData1:
                    if row[1] == 'Complete submission text file':
                        url10Q = 'https://www.sec.gov' + row[2]
                        break

                browser.get(url10Q)
                html2 = browser.page_source

                soup2 = BeautifulSoup(html2, 'html.parser')
                text = soup2.text
                try:
                    documents = re.findall("<DOCUMENT>(.*?)</DOCUMENT>", text, re.DOTALL)
                    text_10q = parse_sec_text(text, documents)
                    text_file = open(cik + '_' + date + '.txt', "w")
                    n = text_file.write(text_10q)
                    text_file.close()
                except:
                    print('Fail to fetch results for ' + cik + '_' + date)
    except:
        print('Unable to look up CIK ' + str(cik))